In [1]:
# imports
import sys
import os
import socket
import pandas as pd
import glob
from datetime import datetime

In [2]:
# read in test symptoms
dementia_symptoms_test = pd.read_parquet(os.getcwd() + '/dementia_symptoms_test_data.parquet')
dementia_symptoms_test.head()

,e_patid,eventdate,sysdate,constype,consid,medcode,enttype,data1,data2,data3,data4,data5,data6,data7,data8
904214,206414246,2001-08-06,2002-02-07,2,944142,7742,288,0.0,NaN,0.0,0,NaN,NaN,0.0,None
904242,206414246,2001-08-06,2002-02-07,2,944142,7742,288,0.0,NaN,0.0,0,NaN,NaN,0.0,None
904268,206414246,2001-08-06,2002-02-07,2,944142,7742,288,0.0,NaN,0.0,0,NaN,NaN,0.0,None
4837330,678148246,1997-05-15,2002-02-07,2,369618,1993,288,0.0,NaN,0.0,0,NaN,NaN,0.0,None
4837354,678148246,1997-05-15,2002-02-07,2,369618,1993,288,0.0,NaN,0.0,0,NaN,NaN,0.0,None


In [3]:
# read in referral symptoms
dementia_symptoms_referral = pd.read_parquet(os.getcwd() + '/dementia_symptoms_referral_data.parquet')
dementia_symptoms_referral.head()

,e_patid,eventdate,sysdate,constype,consid,medcode,nhsspec,attendance,urgency
11,10009416,2003-04-04,2009-02-20,4,1067550,2964,0,0,0
38,10010191,2000-01-18,2000-01-20,4,136437,5777,54,1,4
308,100305191,2004-07-01,2004-07-01,4,476673,10448,36,0,2
379,10036782,2011-03-16,2011-03-16,4,168769,1511,0,0,2
386,10036782,2013-03-04,2013-03-04,4,169048,2189,54,0,4


In [4]:
# get unique e_patids for both
e_patids_test = dementia_symptoms_test['e_patid'].unique()
e_patids_referral = dementia_symptoms_referral['e_patid'].unique()

In [5]:
# Replace nan eventdates with sysdates and make date object
dementia_symptoms_test['eventdate'].fillna(dementia_symptoms_test['sysdate'], inplace = True)
dementia_symptoms_referral['eventdate'].fillna(dementia_symptoms_referral['sysdate'], inplace = True)

In [6]:
# get first symptom in test file
first_dementia_code_dates_test = []
for i in e_patids_test:
    first_dementia_code_dates_test += [min(
        dementia_symptoms_test[dementia_symptoms_test['e_patid'] == i]['eventdate'])]

In [7]:
# make df
df_first_dates_test = pd.DataFrame({"e_patid": e_patids_test, "eventdate": first_dementia_code_dates_test})

In [8]:
df_first_dates_test.head()

,e_patid,eventdate
0,206414246,2001-08-06
1,678148246,1997-05-15
2,633529235,1990-11-27
3,641118235,1993-08-06


In [9]:
# merge with test file to get first symptom row
first_dementia_symptoms_test = df_first_dates_test.merge(
    right = dementia_symptoms_test, on = ['e_patid', 'eventdate'], how = 'inner')

In [10]:
first_dementia_symptoms_test.shape[0]

10

In [11]:
dementia_symptoms_test.shape[0]

10

In [12]:
# save to parquet
first_dementia_symptoms_test.to_parquet(os.getcwd() + '/first_dementia_symptoms_in_primary_test.parquet')

In [13]:
# get first symptom in referral file
first_dementia_code_dates_referral = []
for i in e_patids_referral:
    first_dementia_code_dates_referral += [min(
        dementia_symptoms_referral[dementia_symptoms_referral['e_patid'] == i]['eventdate'])]

In [14]:
# save to df
df_first_dates_referral = pd.DataFrame({"e_patid": e_patids_referral, "eventdate": first_dementia_code_dates_referral})

In [15]:
df_first_dates_referral.head()

,e_patid,eventdate
0,10009416,2003-04-04
1,10010191,2000-01-18
2,100305191,2004-07-01
3,10036782,2007-06-19
4,10040816,2018-02-27


In [16]:
# merge as above with referral file file
first_dementia_symptoms_referral = df_first_dates_referral.merge(
    right = dementia_symptoms_referral, on = ['e_patid', 'eventdate'], how = 'inner')

In [17]:
first_dementia_symptoms_referral.shape[0]

88753

In [18]:
dementia_symptoms_referral.shape[0]

122785

In [19]:
# save to parquet file
first_dementia_symptoms_referral.to_parquet(os.getcwd() + '/first_dementia_symptoms_in_primary_referral.parquet')